In [3]:
# Imports
import requests
import json
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
    Trainer, 
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import time
import requests

In [4]:
#Setting up the connection
API_BASE = "http://194.171.191.227:30080"
token = "sk-128603fe9de54d68ab264c3ee96067b5"
# List models
header = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
response = requests.get(f"{API_BASE}/api/models", headers=header)
print(json.dumps(response.json(), indent=2))

{
  "data": [
    {
      "id": "llama3.2:3b",
      "name": "llama3.2:3b",
      "object": "model",
      "created": 1743610271,
      "owned_by": "ollama",
      "ollama": {
        "name": "llama3.2:3b",
        "model": "llama3.2:3b",
        "modified_at": "2025-02-27T16:41:08.841805321Z",
        "size": 2019393189,
        "digest": "a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72",
        "details": {
          "parent_model": "",
          "format": "gguf",
          "family": "llama",
          "families": [
            "llama"
          ],
          "parameter_size": "3.2B",
          "quantization_level": "Q4_K_M"
        },
        "urls": [
          0
        ]
      },
      "info": {
        "id": "llama3.2:3b",
        "user_id": "2b6ec3d8-8ef1-452b-9d4e-4305d4d23c42",
        "base_model_id": null,
        "name": "llama3.2:3b",
        "params": {},
        "meta": {
          "profile_image_url": "/static/favicon.png",
          "description": nul

In [5]:
EMOTIONS = ['happiness', 'sadness', 'anger', 'surprise', 'fear', 'disgust', 'neutral']

SYSTEM_PROMPT = f'''
You are an expert in recognizing emotions from text. The input will be a list of sentences, and for each sentence, you need to predict the most appropriate emotion from the following list:

{", ".join(EMOTIONS)}

Each emotion is defined as:

Happiness: Feeling of joy, contentment, or satisfaction.

Sadness: A feeling of sorrow, grief, or unhappiness.

Anger: A strong feeling of displeasure or hostility.

Disgust: A feeling of revulsion or strong disapproval.

Fear: A feeling of anxiety or concern about potential danger or harm.

Surprise: A feeling of astonishment or unexpectedness.

Neutral: No strong emotion, often used for informational or factual statements.

Example 1:

Sentence: "I just got promoted at work!"

Predicted Emotion: Happiness

Example 2:

Sentence: "I'm so frustrated with how things are going at home."

Predicted Emotion: Anger

Example 3:

Sentence: "The movie was a bit boring, but not terrible."

Predicted Emotion: Neutral

Example 4:

Sentence: "I can't believe I forgot my best friend's birthday!"

Predicted Emotion: Sadness

Example 5:

Sentence: "The sudden loud noise made me jump!"

Predicted Emotion: Surprise

Instructions for Ambiguous Cases:

If the emotion is unclear or not explicitly stated, return the closest emotion from the list based on context.

Choose the emotion that most accurately reflects the sentiment of the sentence.

Please provide only the emotion as your output for each sentence, and ensure the output is one of the emotions from the provided list.
'''

In [6]:
def emotion_classifier(user_prompt):
    url = 'http://194.171.191.227:30080/api/chat/completions'

    # Prepare the prompt for the API call
    prompt = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
    ]

    # Make the API request
    response = requests.post(url, headers=header, json={"model": "llama3.3:latest", "messages": prompt})

    # Parse the assistant's response
    assistant_response = response.json()

    # Access the content of the assistant's response
    content = assistant_response['choices'][0]['message']['content']

    # Extract only the emotion, using a list of known emotions
    emotions = ['happiness', 'sadness', 'anger', 'surprise', 'fear', 'disgust', 'neutral']
    
    # Filter content to include only the emotions you care about
    for emotion in emotions:
        if emotion in content.lower():
            return emotion  # Return the first matching emotion

    return "No emotion detected"  # In case no emotion is found

In [7]:
def map_emotion(emotion_label):
    if isinstance(emotion_label, str):  # Ensure it's a string
        for core_emotion, subcategories in emotion_mapping.items():
            if emotion_label.lower() in subcategories:
                return core_emotion
    return emotion_label  # Keep original value if not found

emotion_mapping = {
    "happiness": [
        "joy", "optimism", "approval", "pride", "gratitude", "confidence",
        "satisfaction", "hope", "love", "excitement", "caring", "relief", "admiration",
        "amusement", "anticipation", "encouragement", "desire", "happiness"
    ],
    "sadness": [
        "disappointment", "nostalgia", "remorse", "pain", "stress", "regret",
        "resignation", "despair", "confusion", "uncertainty", "sadness"
    ],
    "anger": [
        "anger", "annoyance", "disapproval", "frustration", "disbelief", "warning",
        "rejection"
    ],
    "disgust": [
        "disgust"
    ],
    "fear": [
        "fear", "nervousness", "worry", "anxiety", "doubt", "insecurity", "urgency",
        "panic"
    ],
    "surprise": [
        "surprise", "realization", "shock", "amazement", "wonder"
    ],
    "neutral": [
        "neutral", "mixed", "trust", "agreement", "instruction", "suggestion",
        "request", "confirmation", "acceptance", "reassurance", "clarification",
        "understanding", "certainty"
    ]
}

In [8]:
# List of user prompts
user_prompt = [
    'The weather is nice today', 
    'I am feeling very sad', 
    'I am so excited for the party', 
    'I am so scared of spiders', 
    'I am so angry at you', 
    'I am so disgusted by the food', 
    'I am so surprised by the news', 
    'I hate everyone',
    'John is a very nice person, even though he is a bit annoying sometimes'
]

# Call the emotion classifier function for each user prompt
for prompt in user_prompt:
    predicted_emotions = emotion_classifier(prompt)  # Only get predicted emotions

    # Output the results for each prompt
    if predicted_emotions is not None:
        print(predicted_emotions)  # Just print the predicted emotions

happiness
sadness
happiness
fear
anger
disgust
surprise
anger
neutral


In [11]:
# Load the CSV file
file_path = "group_25.csv"  
df = pd.read_csv(file_path)

# Display the first few rows
print(df.head())

     Start Time      End Time  \
0  00:00:00,000  00:00:03,000   
1  00:00:03,000  00:00:09,000   
2  00:00:11,000  00:00:19,000   
3  00:00:19,000  00:00:20,000   
4  00:00:20,000  00:00:23,000   

                                            Sentence  \
0                                      BOH-AM-Z 200%   
1                   ONLY ACTIVATES WHEN MEN'T ACTIVE   
2                                         LEST ARGUE   
3                                               OUCH   
4  BECAUSE PEP rapt is my favorite boo and is 100...   

                                  Corrected Sentence  \
0                                      BOH-AM-Z 200%   
1                   ONLY ACTIVATES WHEN MEN'T ACTIVE   
2                                         LEST ARGUE   
3                                               OUCH   
4  BECAUSE PEP rapt is my favorite boo and is 100...   

                                         Translation    Emotion Intensity  
0                                      Boh-am-Z 200

In [12]:
df["Mapped_Emotion"] = df["Emotion"].astype(str).apply(map_emotion)

In [13]:
# Drop NaN values and empty strings
df = df.dropna(subset=["Translation"])
df = df[df["Translation"].str.strip() != ""]

# Convert to list for classification
sentences = df["Translation"].tolist()

# Process in batches to avoid infinite execution
batch_size = 10  # Adjust batch size as needed

predicted_emotions = []
for i in range(0, len(sentences), batch_size):
    batch = sentences[i : i + batch_size]
    
    for sentence in batch:
        emotion = emotion_classifier(sentence)
        predicted_emotions.append(emotion)
        time.sleep(1)  # Prevent API overload
    
    # Print progress after each batch
    print(f"Processed {i + len(batch)} of {len(sentences)} sentences")

# Attach predictions to DataFrame
df["Predicted_Emotion"] = predicted_emotions[:len(df)]  # Ensure matching lengths
print("Emotion classification completed.")

Processed 10 of 2012 sentences
Processed 20 of 2012 sentences
Processed 30 of 2012 sentences
Processed 40 of 2012 sentences
Processed 50 of 2012 sentences
Processed 60 of 2012 sentences
Processed 70 of 2012 sentences
Processed 80 of 2012 sentences
Processed 90 of 2012 sentences
Processed 100 of 2012 sentences
Processed 110 of 2012 sentences
Processed 120 of 2012 sentences
Processed 130 of 2012 sentences
Processed 140 of 2012 sentences
Processed 150 of 2012 sentences
Processed 160 of 2012 sentences
Processed 170 of 2012 sentences
Processed 180 of 2012 sentences
Processed 190 of 2012 sentences
Processed 200 of 2012 sentences
Processed 210 of 2012 sentences
Processed 220 of 2012 sentences
Processed 230 of 2012 sentences
Processed 240 of 2012 sentences
Processed 250 of 2012 sentences
Processed 260 of 2012 sentences
Processed 270 of 2012 sentences
Processed 280 of 2012 sentences
Processed 290 of 2012 sentences
Processed 300 of 2012 sentences
Processed 310 of 2012 sentences
Processed 320 of 

In [14]:
# Display the DataFrame with the new column
print(df.head())

     Start Time      End Time  \
0  00:00:00,000  00:00:03,000   
1  00:00:03,000  00:00:09,000   
2  00:00:11,000  00:00:19,000   
3  00:00:19,000  00:00:20,000   
4  00:00:20,000  00:00:23,000   

                                            Sentence  \
0                                      BOH-AM-Z 200%   
1                   ONLY ACTIVATES WHEN MEN'T ACTIVE   
2                                         LEST ARGUE   
3                                               OUCH   
4  BECAUSE PEP rapt is my favorite boo and is 100...   

                                  Corrected Sentence  \
0                                      BOH-AM-Z 200%   
1                   ONLY ACTIVATES WHEN MEN'T ACTIVE   
2                                         LEST ARGUE   
3                                               OUCH   
4  BECAUSE PEP rapt is my favorite boo and is 100...   

                                         Translation    Emotion Intensity  \
0                                      Boh-am-Z 20

In [15]:
# Replace 'No emotion detected' with 'Neutral' in Predicted_Emotion
df['Predicted_Emotion'] = df['Predicted_Emotion'].replace('No emotion detected', 'neutral')

# Replace the capital letter 
df['Predicted_Emotion'] = df['Predicted_Emotion'].replace('Neutral', 'neutral')

# Replace 'nan' with 'neutral' in Mapped_Emotion
df['Mapped_Emotion'] = df['Mapped_Emotion'].replace('nan', 'neutral')

# Replace 'embarrassment' with 'sadness' in Mapped_Emotion
df['Mapped_Emotion'] = df['Mapped_Emotion'].replace('embarrassment', 'sadness')

# Replace 'curiosity' with 'neutral' in Mapped_Emotion
df['Mapped_Emotion'] = df['Mapped_Emotion'].replace('curiosity', 'neutral')


In [16]:
# Get unique values in both 'Mapped_Emotion' and 'Predicted_Emotion' columns
unique_mapped_emotions = df['Mapped_Emotion'].unique()
unique_predicted_emotions = df['Predicted_Emotion'].unique()

# Display the unique values
print("Unique Mapped_Emotion values:", unique_mapped_emotions)
print("Unique Predicted_Emotion values:", unique_predicted_emotions)

Unique Mapped_Emotion values: ['sadness' 'anger' 'neutral' 'happiness' 'surprise' 'disgust' 'fear']
Unique Predicted_Emotion values: ['surprise' 'neutral' 'anger' 'happiness' 'sadness' 'disgust' 'fear']


In [17]:
# Initialize label encoder
label_encoder = LabelEncoder()

# Encode Corrected_Emotion and Predicted_Emotion
df['Corrected_Emotion_Label'] = label_encoder.fit_transform(df['Mapped_Emotion'])
df['Predicted_Emotion_Label'] = label_encoder.transform(df['Predicted_Emotion'])

In [18]:
# Calculate F1 Score
f1 = f1_score(df['Corrected_Emotion_Label'], df['Predicted_Emotion_Label'], average='micro')
print(f"F1 Score: {f1:.4f}")

F1 Score: 0.6173


In [19]:
# Generate correlation matrix
correlation_matrix = df[['Corrected_Emotion_Label', 'Predicted_Emotion_Label']].corr()
print("Correlation Matrix:")
print(correlation_matrix)

Correlation Matrix:
                         Corrected_Emotion_Label  Predicted_Emotion_Label
Corrected_Emotion_Label                 1.000000                 0.381608
Predicted_Emotion_Label                 0.381608                 1.000000
